In [20]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

import os
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-


In [58]:
import anthropic
import gradio as gr
import json
from datetime import datetime

class TravelPlanner:
    def __init__(self):
        self.client = anthropic.Anthropic()

    def generate_travel_recommendations(self, travel_details):
        try:
            system_prompt = """
            Jesteś asystentem podróży AI dostarczającym spersonalizowane rekomendacje dotyczące podróży.
            Przeanalizuj preferencje podróżnego i stwórz szczegółowy, dopasowany plan podróży.
            Weź pod uwagę cel podróży, czas trwania, rodzaj podróży, budżet i inne szczegółowe szczegóły.
            Twoją walutą jest PLN.
            """
            
            response = self.client.messages.create(
                model="claude-3-5-haiku-20241022",
                max_tokens=2000,
                system=system_prompt,
                messages=[
                    {
                        "role": "user", 
                        "content": f"Wygeneruj szczegółowy plan podróży z następującymi parametrami: {json.dumps(travel_details)}"
                    }
                ]
            )
            return response.content[0].text
        except Exception as e:
            return f"Błąd podczas generowania rekomendacji: {str(e)}"

    def chat_with_travel_assistant(self, message, travel_details, chat_history):
        try:
            system_prompt = f"""
            Jesteś asystentem podróży AI, który pomaga użytkownikowi w podróży.
            Szczegóły obecnego planu podróży: {json.dumps(travel_details)}
            
            Odpowiadaj na pytania i prośby związane z podróżą, 
            bazując na wcześniejszych informacjach i wygenerowanym planie.
            Bądź pomocny, precyzyjny i dostosuj odpowiedź do kontekstu podróży.
            """
            
            # Prepare messages for API call
            messages = []
            
            for past_message in chat_history:
                messages.append({
                    "role": past_message["role"], 
                    "content": past_message["content"]
                })
            
            messages.append({"role": "user", "content": message})
            
            response = self.client.messages.create(
                model="claude-3-5-haiku-20241022",
                max_tokens=1000,
                system=system_prompt,
                messages=messages
            )
            
            # Update chat history
            chat_history.append({
                "role": "user", 
                "content": message
            })
            chat_history.append({
                "role": "assistant", 
                "content": response.content[0].text
            })
            
            return chat_history, "", chat_history
        except Exception as e:
            error_message = {
                "role": "assistant", 
                "content": f"Błąd podczas rozmowy z asystentem: {str(e)}"
            }
            chat_history.append(error_message)
            return chat_history, "", chat_history

    def process_travel_details(
        self, destination, min_days, max_days, travel_month, 
        travelers, travel_type, budget_per_person, multi_city, 
        car_rental, daily_driving_limit
    ):
        # Calculate total budget
        total_budget = budget_per_person * travelers
        
        travel_details = {
            "destination": destination,
            "duration": f"{min_days}-{max_days} dni",
            "travel_dates": travel_month,
            "travelers": travelers,
            "travel_type": travel_type,
            "budget_per_person": budget_per_person,
            "total_budget": total_budget,
            "multi_city": multi_city,
            "car_rental": car_rental,
            "daily_driving_distance": daily_driving_limit
        }
        
        recommendations = self.generate_travel_recommendations(travel_details)
        
        chat_history = [
            {
                "role": "assistant", 
                "content": recommendations
            }
        ]
        
        return chat_history, travel_details

    def create_travel_interface(self):
        with gr.Blocks() as demo:
            travel_details = gr.JSON(visible=False)
            chat_history = gr.State([])
            
            with gr.Column():
                gr.Markdown("# 🌍 Planer Podróży")
                
                destination = gr.Textbox(label="Cel podróży", placeholder="Wpisz kraj lub region")
                
                with gr.Row():
                    min_days = gr.Slider(minimum=1, maximum=30, value=3, step=1, label="Minimalna liczba dni")
                    max_days = gr.Slider(minimum=1, maximum=30, value=7, step=1, label="Maksymalna liczba dni")
                
                travel_month = gr.Dropdown(
                    label="Preferowany miesiąc podróży", 
                    choices=[
                        "Styczeń", "Luty", "Marzec", "Kwiecień", "Maj", "Czerwiec", 
                        "Lipiec", "Sierpień", "Wrzesień", "Październik", "Listopad", "Grudzień"
                    ]
                )
                
                travelers = gr.Slider(minimum=1, maximum=10, value=2, step=1, label="Liczba podróżujących")
                
                travel_type = gr.Dropdown(
                    label="Rodzaj podróży", 
                    multiselect=True,
                    choices=[
                        "Turystyka wypoczynkowa", "Aktywny wypoczynek", 
                        "Zwiedzanie historyczne", "Odkrywanie krajobrazów",
                        "Kultura i sztuka", "Doświadczenia kulinarne",
                        "Turystyka przygodowa", "Wakacje na plaży",
                        "Przyroda i zwierzęta"
                    ]
                )
                
                budget_per_person = gr.Number(label="Budżet na osobę (PLN)", precision=0)
                multi_city = gr.Checkbox(label="Zwiedzanie wielu miast")
                
                with gr.Row():
                    car_rental = gr.Checkbox(label="Wypożyczenie samochodu")
                    daily_driving_limit = gr.Slider(
                        minimum=10, maximum=500, value=200, 
                        step=10, label="Maksymalna dzienna odległość w linii prostej (km)"
                    )
                
                submit_btn = gr.Button("Generuj plan podróży", variant="primary")
                
                chatbot = gr.Chatbot(label="Asystent Podróży", height=800, type="messages")
                chat_msg = gr.Textbox(label="Zadaj pytanie")
                chat_submit = gr.Button("Wyślij")
                
            submit_btn.click(
                fn=self.process_travel_details,
                inputs=[
                    destination, min_days, max_days, travel_month, 
                    travelers, travel_type, budget_per_person, multi_city, 
                    car_rental, daily_driving_limit
                ],
                outputs=[chatbot, travel_details]
            )
            
            chat_submit.click(
                fn=self.chat_with_travel_assistant,
                inputs=[chat_msg, travel_details, chat_history],
                outputs=[chatbot, chat_msg, chat_history]
            )
        
        return demo

# Launch the application
planner = TravelPlanner()
demo = planner.create_travel_interface()
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7887
* Running on public URL: https://19831595b762d0f8d1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
